# Исследование алгоритмов

### Проверка и установка рабочей директории, должен быть корень проекта

In [1]:
%pwd

'C:\\Users\\Kuroha\\source\\repos_py\\bauman_final_project\\notebooks'

In [2]:
%cd ..

C:\Users\Kuroha\source\repos_py\bauman_final_project


### Загрузка датасета

In [3]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from src.utils import *

In [4]:
path = get_filepath(DATA_PROCESSED_TRAIN, is_raw=False)
df = pd.read_csv(path, parse_dates=['date'])

In [5]:
df.head().T

,0,1,2,3,4
uid,9386,9386,9386,9386,9386
date,2008-01-01 00:00:00,2008-01-02 00:00:00,2008-01-03 00:00:00,2008-01-04 00:00:00,2008-01-05 00:00:00
latitude,0.312439,0.312439,0.312439,0.312439,0.312439
longitude,0.699807,0.699807,0.699807,0.699807,0.699807
temperature,0.438202,0.314607,0.146067,0.168539,0.325843
pressure,0.414474,0.519737,0.546053,0.546053,0.361842
cloud,1.0,0.0,0.0,0.666667,1.0
wind_spd,0.333333,0.166667,0.0,0.0,0.666667
is_fallback_data,1,1,1,1,1
water_level,138.0,138.0,138.0,138.0,138.0


### Подготовка данных

Необходимо проверить, зависит ли качество моделей от способа кодирования погоды: одной колонкой с наличием осадков, двумя - дождь и снег, либо тремя - дождь, гроза и снег.

Датасеты с разные вариантами кодирования погоды будут именоваться как **v1**, **v2**, **v3**.

In [6]:
df.columns

Index(['uid', 'date', 'latitude', 'longitude', 'temperature', 'pressure',
       'cloud', 'wind_spd', 'is_fallback_data', 'water_level', 'uid_0',
       'uid_1', 'uid_2', 'uid_3', 'uid_4', 'uid_5', 'uid_6', 'uid_7', 'uid_8',
       'uid_9', 'uid_10', 'uid_11', 'uid_12', 'uid_13', 'uid_14', 'year',
       'day_sin', 'day_cos', 'weather_v1_precip', 'weather_v2_rain',
       'weather_snow', 'weather_v3_rain', 'weather_v3_storm', 'north', 'south',
       'west', 'east', 'pressure_upper', 'pressure_lower', 'wind_spd_upper'],
      dtype='object')

Для обучения модели используем данные за 2008-2016 года, а для тестирования - за 2017.

In [7]:
df_X_train = df.loc[(df['date'] < '2017-01-01')]
df_X_test = df.loc[(df['date'] >= '2017-01-01')]
y_train = df_X_train['water_level']
y_test = df_X_test['water_level']

print(f'''Размерность оригинального датасета: {df.shape}
Размерность тренировочного датасета: {df_X_train.shape}
Размерность тестового датасета: {df_X_test.shape}
Размер тестовой выборки: {df_X_test.shape[0] / df_X_train.shape[0] * 100:.2f}%''')

Размерность оригинального датасета: (71159, 40)
Размерность тренировочного датасета: (61535, 40)
Размерность тестового датасета: (9624, 40)
Размер тестовой выборки: 15.64%


In [8]:
def get_all_df(input_df):
    df_all_base = input_df.drop(['date', 'water_level', 'uid'], axis=1)
    df_all_v1 = df_all_base.drop(['weather_v2_rain', 'weather_snow', 'weather_v3_rain', 'weather_v3_storm'], axis=1)
    df_all_v2 = df_all_base.drop(['weather_v1_precip', 'weather_v3_rain', 'weather_v3_storm'], axis=1)
    df_all_v3 = df_all_base.drop(['weather_v1_precip', 'weather_v2_rain'], axis=1)
    return [df_all_v1, df_all_v2, df_all_v3]

df_X_train_all = get_all_df(df_X_train)
df_X_test_all = get_all_df(df_X_test)

### Применение алгоритмов машинного обучения

Для оценки качества моделей будут использованы 2 меры качества: средняя квадратичная ошибка (MSE) и коэффициент детерминации ($R^2$).

In [9]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score

results = {}  # словарь с сохранёнными метриками
algos = {}  # словарь со всеми алгоритмами

In [10]:
# Сохранение модели вместе с оценкой предсказания
def add_result(model, predict, model_name):
    mae_result = round(mae(y_test, predict), 3)
    r2_score_result = round(r2_score(y_test, predict), 3)
    #mae_result = round(mae(y_test, predict), 3)
    #r2_score_result = round(r2_score(y_test, predict), 3)
    print(model_name)
    print(f'MAE: {mae_result}')
    print(f'R2_score: {r2_score_result}')
    print('=' * 20)
    
    
    results[model_name] = {'MAE': mae_result, 'R2_score': r2_score_result, 'model': model}

# Предсказание по заранее обученной модели
def predict_result(model, X_test, model_name):
    predict = model.predict(X_test)    
    add_result(model, predict, model_name)

# Применение алгоритма ко всем вариациям датасета с последующей оценкой качества
def apply_to_all(algo, model_name):
    algos[model_name] = algo
    for i in range(len(df_X_train_all)):
        model = algo(df_X_train_all[i], df_X_test_all[i])        
        predict_result(model, df_X_test_all[i], f'{model_name} (df_v{i+1})')

#### 1. Заглушка

Представим, что модель всегда возвращает средний уровень воды. Полученные ошибки будут использованы для оценки качества реальных моделей.

In [11]:
y_predicted = pd.Series([y_train.mean()] * y_test.shape[0])
y_predicted

0       266.42995
1       266.42995
2       266.42995
3       266.42995
4       266.42995
          ...    
9619    266.42995
9620    266.42995
9621    266.42995
9622    266.42995
9623    266.42995
Length: 9624, dtype: float64

In [12]:
add_result(None, y_predicted, 'Mean')

Mean
MAE: 153.267
R2_score: -0.014


#### 2. Линейная регрессия

In [13]:
from sklearn.linear_model import LinearRegression 

def linear_regression(X_train, _):    
    linear_regression = LinearRegression()
    linear_regression.fit(X_train, y_train)
    
    return linear_regression

In [14]:
apply_to_all(linear_regression, 'Linear Regression')

Linear Regression (df_v1)
MAE: 109.123
R2_score: 0.412
Linear Regression (df_v2)
MAE: 109.105
R2_score: 0.412
Linear Regression (df_v3)
MAE: 109.104
R2_score: 0.412


In [15]:
pd.DataFrame.from_dict(results, orient='index')

,MAE,R2_score,model
Mean,153.267,-0.014,None
Linear Regression (df_v1),109.123,0.412,LinearRegression()
Linear Regression (df_v2),109.105,0.412,LinearRegression()
Linear Regression (df_v3),109.104,0.412,LinearRegression()


#### 3. Дерево решений

In [16]:
from sklearn.tree import DecisionTreeRegressor

def decision_tree(X_train, _):
    decision_tree = DecisionTreeRegressor()
    decision_tree.fit(X_train, y_train)
    
    return decision_tree

In [17]:
apply_to_all(decision_tree, 'Decision Tree')

Decision Tree (df_v1)
MAE: 70.194
R2_score: 0.594
Decision Tree (df_v2)
MAE: 70.497
R2_score: 0.591
Decision Tree (df_v3)
MAE: 70.544
R2_score: 0.591


#### 4. Нейронная сеть

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout 
from tensorflow.keras.losses import mean_absolute_error


def neural_network(X_train, X_test):
    model = Sequential()
    # 0.629
    #model.add(Dense(256, activation='relu', input_shape=[X_train.shape[1]]))
    #model.add(Dense(256, activation='relu'))
    # 0.633
    #model.add(Dense(256, activation='relu', input_shape=[X_train.shape[1]]))
    #model.add(Dense(256, activation='selu'))
    # 0.634
    #model.add(Dense(256, activation='relu', input_shape=[X_train.shape[1]]))
    #model.add(Dense(256, activation='elu'))
    model.add(Dense(256, activation='relu', input_shape=[X_train.shape[1]]))
    model.add(Dense(256, activation='elu'))
    model.add(Dense(1))
    
    model.compile(loss=mean_absolute_error, optimizer='Adam', metrics=['mae'])
    model.fit(X_train, y_train,
              validation_data=(X_test, y_test),
              epochs=10,
              verbose=1)
    
    return model

In [21]:
apply_to_all(neural_network, 'Neural Network')

Epoch 1/10
1923/1923 [==============================] - 2s 1ms/step - loss: 104.4968 - mae: 104.4968 - val_loss: 78.5455 - val_mae: 78.5455
Epoch 2/10
1923/1923 [==============================] - 2s 1ms/step - loss: 87.6646 - mae: 87.6646 - val_loss: 72.2591 - val_mae: 72.2591
Epoch 3/10
1923/1923 [==============================] - 2s 1ms/step - loss: 82.9092 - mae: 82.9092 - val_loss: 73.5905 - val_mae: 73.5905
Epoch 4/10
1923/1923 [==============================] - 2s 1ms/step - loss: 80.8983 - mae: 80.8983 - val_loss: 68.0828 - val_mae: 68.0828
Epoch 5/10
1923/1923 [==============================] - 2s 1ms/step - loss: 79.8850 - mae: 79.8850 - val_loss: 70.4452 - val_mae: 70.4452
Epoch 6/10
1923/1923 [==============================] - 3s 1ms/step - loss: 79.0097 - mae: 79.0097 - val_loss: 73.2567 - val_mae: 73.2567
Epoch 7/10
1923/1923 [==============================] - 2s 1ms/step - loss: 78.0927 - mae: 78.0927 - val_loss: 69.5889 - val_mae: 69.5889
Epoch 8/10
1923/1923 [==========

In [ ]:
predict = neural_network(df_X_train_all[0], df_X_test_all[0])

mae_result = round(mae(y_test, predict), 3)
r2_score_result = round(r2_score(y_test, predict), 3)
print(f'MAE: {mae_result}')
print(f'R2_score: {r2_score_result}')

#### 5. Гребневая регрессия

In [22]:
from sklearn.linear_model import Ridge

def ridge(X_train, _):
    ridge = Ridge()
    ridge.fit(X_train, y_train)
    
    return ridge

In [23]:
apply_to_all(ridge, 'Ridge')

Ridge (df_v1)
MAE: 109.121
R2_score: 0.411
Ridge (df_v2)
MAE: 109.104
R2_score: 0.412
Ridge (df_v3)
MAE: 109.103
R2_score: 0.412


#### 6. Лассо

In [24]:
from sklearn.linear_model import Lasso

def lasso(X_train, _):
    lasso = Lasso()
    lasso.fit(X_train, y_train)
    
    return lasso

In [25]:
apply_to_all(lasso, 'Lasso')

Lasso (df_v1)
MAE: 117.914
R2_score: 0.365
Lasso (df_v2)
MAE: 117.914
R2_score: 0.365
Lasso (df_v3)
MAE: 117.914
R2_score: 0.365


In [26]:
pd.DataFrame.from_dict(results, orient='index')

,MAE,R2_score,model
Mean,153.267,-0.014,None
Linear Regression (df_v1),109.123,0.412,LinearRegression()
Linear Regression (df_v2),109.105,0.412,LinearRegression()
Linear Regression (df_v3),109.104,0.412,LinearRegression()
Decision Tree (df_v1),70.194,0.594,DecisionTreeRegressor()
Decision Tree (df_v2),70.497,0.591,DecisionTreeRegressor()
Decision Tree (df_v3),70.544,0.591,DecisionTreeRegressor()
Neural Network (df_v1),68.692,0.621,<keras.engine.sequential.Sequential object at ...
Neural Network (df_v2),69.629,0.632,<keras.engine.sequential.Sequential object at ...
Neural Network (df_v3),70.580,0.647,<keras.engine.sequential.Sequential object at ...
